In [1]:
import torch

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
device

'cpu'

In [3]:
import pandas as pd

df_train = pd.read_csv('train_FNN.csv')
df_train.head(3)


,ID_code,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,train_0,0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,...,4.4354,3.9642,3.1364,1.6910,18.5227,-2.3978,7.8784,8.5635,12.7803,-1.0914
1,train_1,0,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,...,7.6421,7.7214,2.5837,10.9516,15.4305,2.0339,8.1267,8.7889,18.3560,1.9518
2,train_2,0,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,...,2.9057,9.7905,1.6704,1.6858,21.6042,3.1417,-6.5213,8.2675,14.7222,0.3965


In [4]:
df_train.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Columns: 202 entries, ID_code to var_199
dtypes: float64(200), int64(1), object(1)
memory usage: 308.2+ MB


In [5]:
import numpy as np

features = torch.tensor(df_train.drop(['ID_code', 'target'], axis=1).values.astype(np.float32))
features.shape

torch.Size([200000, 200])

In [6]:
targets = torch.tensor(df_train['target'].values.astype(np.float32))
targets.shape

torch.Size([200000])

In [13]:
from torch.utils.data import DataLoader, random_split, TensorDataset

full_dataset = TensorDataset(features, targets)

train_size = int(0.8 * len(full_dataset))
test_size = len(full_dataset) - train_size
train_dataset, test_dataset = random_split(full_dataset, [train_size, test_size])
print(len(train_dataset))
print(len(test_dataset))

160000
40000


In [8]:
batch_size = 64

train_dataloader = DataLoader(train_dataset, batch_size=batch_size)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"X: {X.shape} {X.dtype}")
    print(f"y: {y.shape} {y.dtype}")
    break

X: torch.Size([64, 200]) torch.float32
y: torch.Size([64]) torch.float32


X: torch.Size([64, 200]) torch.float32: Indicates that the input features (X) have a shape of [64, 200], meaning there are 64 samples in the batch, and each sample has 200 features. The data type is torch.float32.

y: torch.Size([64]) torch.float32: Indicates that the labels (y) have a shape of [64], meaning there is one label for each sample in the batch. The data type is torch.float32.

In [9]:
from torch import nn

class FNN(nn.Module):
  def __init__(self):
    super().__init__()
    self.fc1 = nn.Linear(200, 16) # Fully connected layer
    self.fc2 = nn.Linear(16, 1) # 1 output - binary class

  def forward(self, x):
    # Activation function for the hidden layer
    x = torch.relu(self.fc1(x))
    # Activation function for the output layer
    x = torch.sigmoid(self.fc2(x))
    return x

model = FNN().to(device)
print(model)

FNN(
  (fc1): Linear(in_features=200, out_features=16, bias=True)
  (fc2): Linear(in_features=16, out_features=1, bias=True)
)


In [10]:
loss_fn = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

def to_bin(prob, threshold=0.5):
  return (prob >= threshold).float()

def train(dataloader, model, loss_fn, optimizer):
  size = len(dataloader.dataset)
  model.train()
  for batch, (X, y) in enumerate(dataloader):
    X, y = X.to(device), y.to(device)

    pred = model(X)
    # Squeeze from [64, 1] to [64]
    pred = pred.squeeze()
    loss = loss_fn(pred, y)

    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    if batch % 100 == 0:
      loss, current = loss.item(), (batch + 1) * len(X)
      print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test(dataloader, model, loss_fn):
  size = len(dataloader.dataset)
  num_batches = len(dataloader)
  model.eval()
  test_loss, correct = 0, 0
  with torch.no_grad():
    for X, y in dataloader:
      X, y = X.to(device), y.to(device)
      pred = model(X)
      test_loss += loss_fn(pred.squeeze(), y).item()

      # Squeeze and convert to bin before comparing with validation targets
      pred = to_bin(pred.squeeze())
      y = y.squeeze()

      correct += (pred == y).type(torch.float).sum().item()
  test_loss /= num_batches
  correct /= size
  print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
'Done'

Epoch 1
-------------------------------
loss: 1.568110  [   64/160000]
loss: 0.246174  [ 6464/160000]
loss: 0.275184  [12864/160000]
loss: 0.196581  [19264/160000]
loss: 0.352148  [25664/160000]
loss: 0.257878  [32064/160000]
loss: 0.280501  [38464/160000]
loss: 0.181055  [44864/160000]
loss: 0.300710  [51264/160000]
loss: 0.280762  [57664/160000]
loss: 0.221241  [64064/160000]
loss: 0.166118  [70464/160000]
loss: 0.374735  [76864/160000]
loss: 0.305476  [83264/160000]
loss: 0.282701  [89664/160000]
loss: 0.394051  [96064/160000]
loss: 0.333972  [102464/160000]
loss: 0.331721  [108864/160000]
loss: 0.304490  [115264/160000]
loss: 0.335499  [121664/160000]
loss: 0.313643  [128064/160000]
loss: 0.302646  [134464/160000]
loss: 0.449919  [140864/160000]
loss: 0.358036  [147264/160000]
loss: 0.155894  [153664/160000]
Test Error: 
 Accuracy: 90.0%, Avg loss: 0.299889 

Epoch 2
-------------------------------
loss: 0.332807  [   64/160000]
loss: 0.198308  [ 6464/160000]
loss: 0.269565  [12864

'Done'